In [1]:
!pip list

Package                           Version
--------------------------------- ------------
aiobotocore                       2.7.0
aiohttp                           3.9.3
aioitertools                      0.7.1
aiosignal                         1.2.0
alabaster                         0.7.12
altair                            5.0.1
anaconda-anon-usage               0.4.3
anaconda-catalogs                 0.2.0
anaconda-client                   1.12.3
anaconda-cloud-auth               0.1.4
anaconda-navigator                2.5.2
anaconda-project                  0.11.1
anyio                             4.2.0
appdirs                           1.4.4
archspec                          0.2.1
argon2-cffi                       21.3.0
argon2-cffi-bindings              21.2.0
arrow                             1.2.3
astroid                           2.14.2
astropy                           5.3.4
asttokens                         2.0.5
async-lru                         2.0.4
atomicwrites             

In [1]:
import nltk
from transformers import BertTokenizer, BertModel
import torch
import numpy as np
from data_preprocessing.pdf_extract import pdf_extract
from sklearn.metrics.pairwise import cosine_similarity
from nltk.tokenize import sent_tokenize
from nltk import download

C:\Users\User\anaconda3\Lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [2]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

In [3]:
def embed_text(text, tokenizer, model):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
    outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1).detach().numpy()
    return embeddings

In [4]:
download("punkt")


def split_text(text: str) -> list[str]:
    return sent_tokenize(text)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
def split_text_into_paragraphs(text):
    paragraphs = text.split('\n \n')  # Разделяем текст по двойному переводу строки
    paragraphs = [p.strip().replace('\n', '') for p in paragraphs if
                  p.strip()]  # Убираем лишние пробелы и пустые строки
    
    for paragraph in range(len(paragraphs)):
        if 'Рисунок' in paragraphs[paragraph]:
            paragraphs[paragraph] = ''
    return paragraphs


text = pdf_extract(
    'data/dataset/Для Хакатона/Инструкция D-1C1-1.20.02 Анализ взаиморасчетов по закупкам (1).pdf')

In [6]:
def embed_pars(paragraphs, tokenizer, model):
    paragraph_embeddings = []

    for paragraph in paragraphs:
        inputs = tokenizer(paragraph, return_tensors='pt', truncation=True, padding=True)
        outputs = model(**inputs)
        embedding = outputs.last_hidden_state.mean(dim=1).detach().numpy()
        paragraph_embeddings.append(embedding)

    paragraph_embeddings = np.vstack(paragraph_embeddings)
    return paragraph_embeddings

In [7]:
def semantic_search(query, paragraphs, tokenizer, model):
    query_embedding = embed_text(query, tokenizer, model)
    paragraph_embeddings = embed_pars(paragraphs, tokenizer, model)
    
    similarities = cosine_similarity(query_embedding, paragraph_embeddings)
    most_similar_idx = np.argmax(similarities)
    
    return paragraphs[most_similar_idx], similarities[0][most_similar_idx]

In [12]:
most_similar_paragraph, similarity_score = semantic_search('отчет для контрагентов', text, tokenizer, model)
most_similar_paragraph

'  Инструкция пользователей D - 1C1 - 1.20.02  Анализ взаиморасчетов по закупкам   Автор  ООО "НПЦ "1С"  Инструкция пользователей « D - 1C1 - 1.20.02  Анализ взаиморасчетов по закупкам »  Страница  17  Страниц  18  Версия  2   2.2.1.6  Формирование отчета « Выполнение условий по договорам контрагентов »  Отчет « Выполнение условий по договорам контрагентов » предназначен для контроля поставок, оформленных по договору с контрагентом. Отчет формируется за указанный пользователем период.  В отчет е выводится два вида показателей –  по выполнению поставок в суммовом отношении и в количественном. При этом выводятся те данные, которые были определены в заказе поставщику или заказе клиента, и те реальные данные поступления или отгрузок, которые были про изведены за период, выбранный в отчете.  Отчет может быть сгруппирован по контрагентам и договорам. .  Отчет « Выполнение условий по договорам контрагентов » расположен в разделе ( Закупки \uf0e0 Отчеты  по закупкам \uf0e0 Закупки (га) \uf0e0 

In [9]:
most_similar_paragraph

'  Инструкция пользователей D - 1C1 - 1.20.02  Анализ взаиморасчетов по закупкам   Автор  ООО "НПЦ "1С"  Инструкция пользователей « D - 1C1 - 1.20.02  Анализ взаиморасчетов по закупкам »  Страница  14  Страниц  18  Версия  2    Рисунок  7 : Расположение отчета « Сводная ведомость расчетов » в ИС  Примечание: если на форме «Расчеты с поставщи ками» не отображается отчет «Сводная ведомость расчетов», необходимо перейти в раздел «Настройка» в поиске набрать  «Сводная»  далее  поставить галку «Сводная ведомость расчетов»  '